# EDA

## Questions

1. **What factors most strongly predict box office success — budget, cast popularity, or IMDb ratings?**
2. **Do movies released in summer perform better financially than those in winter?** 
3. **Is there a significant difference between critics’ and audience ratings across genres?**

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib import rcParams
import seaborn as sns
import ast
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
%matplotlib inline

sns.set_theme(style="whitegrid", context="talk")

In [2]:
processed_data_path = Path("../data/processed.csv")
df = pd.read_csv(processed_data_path)
df

,Unnamed: 0,movie_id,title,release_date,budget,revenue_worldwide,runtime,genres,imdb_id,franchise,...,imdb_votes,mpaa_rating,domestic_box_office,rotten_tomatoes_score,metacritic_score,awards_text,month,season,critics_score,audience_score
0,0,8193,Napoleon Dynamite,2004-06-11,-0.731466,46118097,95,['Comedy'],tt0374900,False,...,251071.0,PG,44540956.0,72.0,64.0,10 wins & 23 nominations total,6,Summer,6.80,0.70
1,1,663558,New Gods: Nezha Reborn,2021-02-06,-0.737418,70000000,117,"['Animation', 'Fantasy', 'Action']",tt13269670,True,...,4451.0,TV-14,NaN,80.0,NaN,3 wins & 3 nominations total,2,Winter,8.00,0.68
2,2,614409,To All the Boys: Always and Forever,2021-02-12,-0.737418,0,115,"['Romance', 'Comedy', 'Drama']",tt10676012,True,...,37385.0,TV-14,NaN,79.0,65.0,2 wins & 2 nominations total,2,Winter,7.20,0.63
3,3,12,Finding Nemo,2003-05-30,0.661406,940335536,100,"['Animation', 'Family']",tt0266543,True,...,1185437.0,G,380843261.0,99.0,90.0,Won 1 Oscar. 49 wins & 63 nominations total,5,Other,9.45,0.82
4,4,16,Dancer in the Dark,2000-09-01,-0.551404,40061153,140,"['Drama', 'Crime']",tt0168629,False,...,122419.0,R,4184036.0,69.0,63.0,Nominated for 1 Oscar. 34 wins & 48 nomination...,9,Other,6.60,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,796,157544,The Vatican Tapes,2015-07-23,-0.543964,1784763,91,"['Thriller', 'Horror']",tt1524575,False,...,19567.0,PG-13,1784763.0,20.0,38.0,NaN,7,Summer,2.90,0.47
791,797,1900,Traffic,2000-12-27,-0.023125,207500000,147,"['Thriller', 'Drama', 'Crime']",tt0181865,False,...,228102.0,R,124115725.0,93.0,86.0,Won 4 Oscars. 73 wins & 86 nominations total,12,Winter,8.95,0.75
792,798,51052,Arthur Christmas,2011-11-10,-0.588607,151300000,97,"['Drama', 'Animation', 'Family', 'Comedy']",tt1430607,False,...,70891.0,PG,46462469.0,93.0,69.0,Nominated for 2 BAFTA 3 wins & 23 nominations ...,11,Other,8.10,0.71
793,799,1903,Vanilla Sky,2001-12-14,0.274497,203388341,136,"['Mystery', 'Romance', 'Science Fiction']",tt0259711,False,...,299078.0,R,100618344.0,41.0,45.0,Nominated for 1 Oscar. 5 wins & 34 nominations...,12,Winter,4.30,0.69


## Dataset Overview
Inspect schema, summary statistics, and missing values before plotting.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 795 entries, 0 to 794
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             795 non-null    int64  
 1   movie_id               795 non-null    int64  
 2   title                  795 non-null    object 
 3   release_date           795 non-null    object 
 4   budget                 795 non-null    float64
 5   revenue_worldwide      795 non-null    int64  
 6   runtime                795 non-null    int64  
 7   genres                 795 non-null    object 
 8   imdb_id                795 non-null    object 
 9   franchise              795 non-null    bool   
 10  cast_popularity_mean   795 non-null    float64
 11  cast_popularity_max    795 non-null    float64
 12  director_popularity    795 non-null    float64
 13  original_language      795 non-null    object 
 14  imdb_rating            795 non-null    float64
 15  imdb_v

In [4]:
overview_stats = df.describe(include='all').transpose().round(2)
overview_stats.head(20)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,795.0,NaN,NaN,NaN,400.188679,231.709439,0.0,199.5,400.0,600.5,800.0
movie_id,795.0,NaN,NaN,NaN,227460.943396,316234.483268,12.0,8276.5,34283.0,402381.0,1376434.0
title,795,794,Monster,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
release_date,795,732,2000-12-22,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
budget,795.0,NaN,NaN,NaN,0.0,1.0,-0.737418,-0.73593,-0.439796,0.378665,6.55284
revenue_worldwide,795.0,NaN,NaN,NaN,173332127.686792,268012579.271297,0.0,207836.5,60780981.0,210823015.0,2068223624.0
runtime,795.0,NaN,NaN,NaN,109.987421,30.446507,3.0,93.0,106.0,121.0,585.0
genres,795,406,['Drama'],30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
imdb_id,795,795,tt0374900,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
franchise,795,2,False,460,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Q1: What factors most strongly predict box office success?
Analyzing correlations between revenue and key predictors: budget, cast popularity, and IMDb rating.

## Corilation Analysis
compute correlations between Revenue worldwide, Budget, Cast Popularity and IMDB Rating

In [5]:
df[['revenue_worldwide', 'budget',
    'cast_popularity_mean', 'imdb_rating']].corr()


,revenue_worldwide,budget,cast_popularity_mean,imdb_rating
revenue_worldwide,1.000000,0.768441,0.388949,0.253140
budget,0.768441,1.000000,0.442348,0.150306
cast_popularity_mean,0.388949,0.442348,1.000000,0.167306
imdb_rating,0.253140,0.150306,0.167306,1.000000


### Answer Q1
**Budget is the strongest predictor of box office success** with a correlation of **0.77** with worldwide revenue.

| Factor | Correlation with Revenue |
|--------|--------------------------|
| Budget | 0.77 (Strong) |
| Cast Popularity | 0.39 (Moderate) |
| IMDb Rating | 0.25 (Weak) |

This suggests that higher production budgets (which enable better marketing, special effects, and star power) are far more predictive of financial success than critical acclaim or cast popularity alone.

## Q2: Do movies released in summer perform better financially than those in winter?
Comparing average worldwide revenue between Summer and Winter releases.

In [6]:
q2_df = df[(df['season'].isin(['Summer', 'Winter'])) & (df['revenue_worldwide'] > 0)].copy()

In [7]:
seasonal_comparison = q2_df.groupby('season')['revenue_worldwide'].mean().sort_values(ascending=False)

print("Average Worldwide Revenue by Season:")
print(seasonal_comparison)

Average Worldwide Revenue by Season:
season
Summer    2.341969e+08
Winter    2.190730e+08
Name: revenue_worldwide, dtype: float64


### Answer Q2
**Yes, summer movies perform better financially than winter releases.**

| Season | Average Worldwide Revenue |
|--------|---------------------------|
| Summer | $234.2 million |
| Winter | $219.1 million |

Summer releases earn approximately **$15 million more** on average (~7% higher). This aligns with industry knowledge that summer blockbuster season (May-August) attracts larger audiences due to school vacations and favorable weather for moviegoing.

## Q3: Is there a significant difference between critics' and audience ratings across genres?
Comparing audience and critics scores by genre to identify where opinions diverge most.

In [8]:
# Parse genres from string representation to list, then explode
q3_df = df.copy()
q3_df['genres'] = q3_df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
q3_df = q3_df.explode('genres').rename(columns={'genres': 'Genre'})
q3_df = q3_df.dropna(subset=['Genre'])

In [9]:
genre_comparison = q3_df.groupby('Genre')[['audience_score', 'critics_score']].mean().dropna()

In [10]:
genre_comparison['difference'] = genre_comparison['audience_score'] - genre_comparison['critics_score']

print("Ratings Comparison by Genre (Audience vs Critics):")
print(genre_comparison.sort_values(by='difference', ascending=False))

Ratings Comparison by Genre (Audience vs Critics):
                 audience_score  critics_score  difference
Genre                                                     
Horror                 0.589286       5.229630   -4.640344
Romance                0.644962       5.533193   -4.888231
Comedy                 0.631031       5.520045   -4.889015
Mystery                0.644091       5.625424   -4.981333
Thriller               0.648173       5.668956   -5.020783
Action                 0.660285       5.795370   -5.135086
Crime                  0.672920       5.828241   -5.155320
Music                  0.654706       5.950000   -5.295294
Western                0.720000       6.050000   -5.330000
War                    0.715217       6.045238   -5.330021
Science Fiction        0.661150       6.011881   -5.350731
Adventure              0.661512       6.021978   -5.360466
Fantasy                0.659416       6.050855   -5.391439
Family                 0.641570       6.086842   -5.445272
Drama

### Answer Q3
**Yes, there is a significant difference — critics consistently rate movies higher than audiences across all genres.**

Key findings:
- **Smallest gap**: Horror (-4.64) — critics and audiences agree most on horror films
- **Largest gap**: TV Movies (-7.12) and Documentaries (-6.98) — critics rate these much higher than audiences
- **Pattern**: Critics favor prestige genres (Documentary, Animation, Drama, History) while audiences show more skepticism

The negative differences indicate that critics' scores (on a ~10-point scale) are consistently higher than audience scores (on a 0-1 scale normalized). This suggests critics value artistic merit while audiences may weight entertainment value differently.